Karamcheti, S., Nair, S., Balakrishna, A., Liang, P., Kollar, T., & Sadigh, D. (2024). [Prismatic VLMs: Investigating the Design Space of Visually-Conditioned Language Models](https://arxiv.org/abs/2402.07865). arXiv preprint arXiv:2402.07865.

**Abstract**
<!-- Visually-conditioned language models (VLMs) have seen growing adoption in applications such as visual dialogue, scene understanding, and robotic task planning; adoption that has fueled a wealth of new models such as LLaVa, InstructBLIP, and PaLI-3. Despite the volume of new releases, key design decisions around image preprocessing, architecture, and optimization are under-explored, making it challenging to understand what factors account for model performance − a challenge further complicated by the lack of objective, consistent evaluations. To address these gaps, we first compile a suite of standardized evaluations spanning visual question answering, object localization, and challenge sets that probe properties such as hallucination; evaluations that provide fine-grained insight to VLM capabilities. Second, we rigorously investigate VLMs along key design axes, including pretrained visual representations and training from base vs. instruct-tuned language models, amongst others. We couple our analysis with three resource contributions: (1) a unified framework for evaluating VLMs, (2) optimized, flexible training code, and (3) checkpoints for all models, including a family of VLMs at the 7-13B scale that strictly outperform InstructBLIP and LLaVa v1.5, the state-of-the-art in open VLMs. -->
视觉条件语言模型 (VLMs) 已在视觉对话、场景理解和机器人任务规划等应用程序中得到越来越广泛的采用; 这种应用催生了大量新模型, 例如 LLaVa、InstructBLIP 和 PaLI-3。尽管新发布数量众多, 但围绕图像预处理、架构和优化的关键设计决策尚未得到充分探索, 使得理解哪些因素影响模型性能变得具有挑战性——缺乏客观、一致的评估使这一挑战更加复杂。为了解决这些差距, 我们首先编制了一套标准化评估, 涵盖视觉问答、物体定位和探索幻觉等属性的挑战集; 评估提供对VLM能力的细粒度洞察。其次, 我们沿关键设计轴严格调查 VLM, 包括预训练的视觉表征和从基础语言模型与指令调整语言模型进行的训练等。我们将我们的分析与三个资源贡献相结合: (1) 用于评估 VLMs 的统一框架, (2) 优化的、灵活的训练代码, (3) 所有模型的检查点, 包括 7-13B 规模的 VLMs 系列, 其严格优于开源 VLMs 中最先进的 InstructBLIP 和 LLaVa v1.5。

# Introduction
<!-- Visually-conditioned language models (VLMs) generate natural language responses from image input and text prompts, providing a general, expressive interface for a growing spectrum of applications – grounded chat (Li et al., 2023c; Gong et al., 2023), visual programming (Sur'is et al., 2023; Subramanian et al., 2023), robotic control (Driess et al., 2023; Brohan et al., 2023), etc. This broad adoption is fueled by a recent paradigm shift in how we develop VLMs; eschewing the complex architectures and training objectives of prior work (Tan & Bansal, 2019; Li et al., 2022; 2023b), new VLMs adopt a simple approach, treating patch features from pretrained visual backbones (e.g., CLIP; Radford et al., 2021) as tokens that can be projected into the input space of a language model (LM). This "patch-as-token" approach enables training with a simple objective – next-token prediction – and allows us to harness the ecosystem of powerful LMs such as Llama-2 and Mistral (Touvron et al., 2023; Jiang et al., 2023), along with the tools to efficiently train them (e.g., FSDP; Zhao et al., 2023). This combination has fueled the rapid development and release of models such as LLaVa v1.5, and PALI-3 that adopt the same underlying recipe, while varying individual ingredients such as the choice of pretrained components, data, or optimization procedure (Liu et al., 2023b; Chen et al., 2023b). -->
视觉条件语言模型 (VLMs) 从图像输入和文本提示生成自然语言响应, 为日益广泛的应用提供通用且富有表现力的接口——基础聊天 (Li et al., 2023c; Gong et al., 2023)、可视化编程 (Sur'is et al., 2023; Subramanian et al., 2023)、机器人控制 (Driess et al., 2023; Brohan et al., 2023) 等。最近我们开发 VLM 方式的范式转变推动了这种广泛采用; 避开先前工作的复杂架构和训练目标 (Tan & Bansal, 2019; Li et al., 2022; 2023b), 新的 VLMs 采用简单的方法, 将预训练视觉 backbones (e.g., CLIP; Radford et al., 2021) 的 patch 特征视为词元, 它可以被投影到语言模型 (LM) 的输入空间。这种"patch 即词元"方法能够以简单的目标(下一个词元预测)进行训练, 并允许我们利用强大的 LMs (例如 Llama-2 和 Mistral)的生态系统 (Touvron et al., 2023; Jiang et al., 2023), 以及有效训练它们的工具(例如 FSDP; Zhao et al., 2023)。这种结合推动了 LLaVa v1.5 和 PALI-3 等模型的快速开发和发布, 这些模型采用相同的底层方法, 同时改变各个成分, 例如预训练组件、数据或优化程序的选择 (Liu et al., 2023b; Chen et al., 2023b)。

<!-- Unfortunately, existing approaches only cover a sliver of the design space around building and training VLMs, without thoroughly evaluating the impact of given choices on downstream capabilities. This motivates the key question of this work: what are the key design decisions that influence VLM capabilities and downstream use? To provide answers to this question, we first need a way to **thoroughly evaluate** the strengths and weaknesses of a given model. Doing this effectively requires compiling a standardized evaluation suite comprised of tasks that are diverse and objective; crucially, these tasks should allow for probing specific capabilities such as spatial reasoning, out-of-distribution generalization, and commonsense understanding, amongst others. Second, we need to **rigorously explore** different VLM design axes, not only to build a concrete set of recommendations, but to tie individual choices to downstream performance. -->
不幸的是, 现有的方法仅涵盖一小部分(围绕构建和训练 VLMs 的)设计空间, 没有彻底评估给定的选择对下游能力的影响。这激发了这项工作的关键问题: 影响 VLM 能力和下游使用的关键设计决策是什么? 为了回答这个问题, 我们首先需要一种方法以**彻底评估**给定模型的优势和劣势。有效地做到这一点, 需要编制一个标准化的评估套件, 包含多样化和客观的任务; 至关重要的是, 这些任务应该允许探索特定能力, 例如空间推理、分布外泛化和常识理解等。其次, 我们需要**严格探索**不同的 VLM 设计轴, 不仅要建立一套具体的建议, 而且要将个人选择与下游绩效联系起来。

<!-- This work addresses these axes through four contributions. First, to provide fine-grained insight into VLM capabilities, **we compile a standardized evaluation suite** comprised of twelve benchmarks from the vision-and-language literature, including four tasks spanning visual question answering (Bigham et al., 2010; Goyal et al., 2017; Hudson & Manning, 2019; Singh et al., 2019), four tasks spanning object localization (Kazemzadeh et al., 2014; Yu et al., 2016; Wang et al., 2021), and four challenge tasks evaluating fine-grained spatial reasoning, hallucination, and diagram understanding (Acharya et al., 2018; Liu et al., 2022; Li et al., 2023d; Kembhavi et al., 2016). Second, we develop an optimized and modular codebase for VLM training that emphasizes flexibility, allowing users to easily swap in pretrained components, optimization procedures, data, and more (Fig. 2; right). Third, we use these resource contributions to perform **targeted experiments exploring four key design axes** (Fig. 2; left): 1) optimization procedure, 2) image processing and visual representations, 3) language models, and 4) scaling training time and data. We identify a number of insights; for example, we find that multi-stage training procedures adopted by existing work can be eliminated without impact on performance, reducing compute costs by 20-25%. We also find that fused visual backbones that merge features from different backbones such as CLIP (Radford et al., 2021) and DINOv2 (Oquab et al., 2023) lead to more performant VLMs across the board. Finally, we consolidate our findings and train a family of models – PRISMs – at the 7B/13B scale that **strictly outperform state-of-the-art open VLMs** such as InstructBLIP and LLaVa v1.5. -->
这项工作通过四个贡献解决这些(设计)轴。首先, 为了提供对 VLM 能力的细粒度洞察, **我们编制了一个标准化评估套件**, 由来自视觉和语言文献的 12 个基准组成, 包括四个视觉问答的任务(Bigham et al., 2010; Goyal et al., 2017; Hudson & Manning, 2019; Singh et al., 2019), 四个物体定位的任务(Kazemzadeh et al., 2014; Yu et al., 2016; Wang et al., 2021), 以及四个评估细粒度空间推理、幻觉和图表理解的挑战任务(Acharya et al., 2018; Liu et al., 2022; Li et al., 2023d; Kembhavi et al., 2016)。第二, **我们开发了一个优化的且模块化的 VLM 训练代码库**, 它强调灵活性, 允许用户轻松地替换预训练组件、优化程序、数据等([图2](#fig.2); 右)。第三, 我们使用这些资源贡献进行**有针对性的实验, 探索四个关键设计轴**([图2](#fig.2); 左): 1) 优化程序, 2) 图像处理和视觉表征, 3) 语言模型, 4) 缩放训练时间和数据。我们发现了一些见解; 例如, 我们发现可以淘汰现有工作采用的多阶段训练程序, 不会影响性能, 计算成本降低 20-25%。我们还发现, 融合视觉 backbones 将全面提高 VLM 的性能, 该 backbones 融合了不同 backbones 的特征, 例如 CLIP (Radford et al., 2021) 和 DINOv2 (Oquab et al., 2023)。最后, 我们整合我们的发现, 并在 7B/13B 规模上训练了一系列模型(PRISM)<sup>[1](#superscript.1)</sup>, 其**严格优于最先进的开源 VLMs**, 例如 InstructBLIP 和 LLaVa v1.5。

<!-- We release our optimized training codebase, evaluation suite, and checkpoints for all models trained as part of this work. -->
> <sup>1</sup><span id='superscript.1'></span> 作为这项工作的一部分, 我们发布了优化的训练代码库、评估套件和所有训练模型的检查点。<br> https://github.com/TRI-ML/prismatic-vlms, https://github.com/TRI-ML/vlm-evaluation

todo: 图2
<!-- Figure 2. **Exploring VLM Design Axes**. We explore four key design axes for developing VLMs: 1) optimization procedure, 2) image processing and pretrained visual representations, 3) language models, and 4) scaling properties around training time and data (left). To enable this exploration, we make a key resource contribution: an open-source, flexible codebase for efficiently training VLMs (right). -->
图2. **探索 VLM 设计轴**。我们探索开发 VLMs 的四个关键设计轴: 1) 优化程序, 2) 图像处理和预训练视觉表征, 3) 语言模型, 4) 围绕训练时间和数据的缩放属性(左)。为了实现这一探索, 我们做出一项关键资源贡献: 一个用于高效训练 VLMs 的开源、灵活的代码库(右)。

# Preliminaries
<!-- To ground our analysis, we require 1) a VLM model architecture, 2) pretraining data, and 3) a training implementation. -->
为了开展分析, 我们需要 1) VLM 模型架构, 2) 预训练数据, 3) 训练实施。

<!-- **Model Architecture**. We adopt the general architecture used by many recent VLMs, such as LLaVa, Qwen-VL, and PaLI-3 (Liu et al., 2023c; Bai et al., 2023; Chen et al., 2023b). These architectures use a (pretrained) visual backbone to map an input image to a sequence of patch features that are then projected individually into the embedding space of an LM. Formally, a VLM takes as input an image $x_\text{img} \in \mathbb{R}^{H \times W}$ and text prompt tokens $u_\text{prompt}$ with arbitrary sequence length $K$. These inputs are then fed to the following components: 1) a visual representation backbone, 2) a vision-language projector, and 3) a language model. -->
**模型架构**。我们采用许多最近的 VLMs 使用的通用架构, 例如 LLaVa, Qwen-VL, 和 PaLI-3 (Liu et al., 2023c; Bai et al., 2023; Chen et al., 2023b)。这些架构使用一个(预训练的)视觉 backbone 将输入图像映射到 patch 特征序列, 然后这些 patch 特征被逐个投影到 LM 的嵌入空间。正式地, VLM 将图像 $x_\text{img} \in \mathbb{R}^{H \times W}$ 和(具有任意序列长度 $K$ 的)文本提示词元 $u_\text{prompt}$ 作为输入。然后这些输入被传输到以下组件: 1) 视觉表征 backbone, 2) 视觉-语言投影器, 3) 语言模型。

<!-- *Visual Representation*. We first process $x_\text{img}$ subject to a visual representation backbone $V_\omega$ that outputs a sequence of features $p_\text{img} \in \mathbb{R}^{L \times h_\text{vision}}$ where $p_\text{img} = V_\omega(x_\text{img})$. As an example, $p_\text{img}$ might be the patch features output by a Vision Transformer (ViT; Dosovitskiy et al., 2021). -->
*视觉表征*。我们首先根据视觉表征 backbone $V_\omega$ 处理 $x_\text{img}$, 该 backbone 输出特征序列 $p_\text{img} \in \mathbb{R}^{L \times h_\text{vision}}$, 其中 $p_\text{img} = V_\omega(x_\text{img})$。例如, $p_\text{img}$ 可能是 Vision Transformer (ViT; Dosovitskiy et al., 2021) 输出的 patch 特征。

<!-- *Vision-Language Projector*. Next, we map $p_\text{img}$ to a sequence of *embeddings* $e_\text{img} \in \mathbb{R}^{L \times h_\text{text}}$ via a learned projector $F_\psi$, where $e_\text{img} = F_\psi(p_\text{img})$. -->
*视觉-语言投影器*。接下来, 我们通过学习到的投影器 $F_\psi$, 将 $p_\text{img}$ 映射到*嵌入*序列 $e_\text{img} \in \mathbb{R}^{L \times h_\text{text}}$, 其中 $e_\text{img} = F_\psi(p_\text{img})$。

<!-- *Language Model*. Finally, we concatenate the sequence $e_\text{img}$ with the text prompt embeddings $e_\text{prompt} = \text{embed}(u_\text{prompt})$, passing the result to the language model. The language model generates output text $u_\text{gen} = \text{LM}_\theta \left( \left[ e_\text{img}; e_\text{prompt} \right] \right)$. -->
*语言模型*。最后, 我们将序列 $e_\text{img}$ 与文本提示嵌入 $e_\text{prompt} = \text{embed}(u_\text{prompt})$ 连接起来, 将结果传递给语言模型。语言模型生成输出文本 $u_\text{gen} = \text{LM}_\theta \left( \left[ e_\text{img}; e_\text{prompt} \right] \right)$。

<!-- The composition $\text{LM}_\theta \left( \left[F_\psi \left( V_\omega(o_\text{rgb}) \right); \text{embed} \left( u_\text{prompt} \right) \right] \right)$ then defines a VLM. Given a triple $\left( x_\text{img}, u_\text{prompt}, \hat{u}_\text{gen} \right)$ during training, we minimize the loss $L \left( \omega, \psi, \theta \right) = - \log p \left( \hat{u}_\text{gen} | x_\text{img}, u_\text{prompt} \right)$ via gradient descent. -->
组合 $\text{LM}_\theta \left( \left[F_\psi \left( V_\omega(o_\text{rgb}) \right); \text{embed} \left( u_\text{prompt} \right) \right] \right)$ 定义了一个 VLM。在训练期间给定一个三元组 $\left( x_\text{img}, u_\text{prompt}, \hat{u}_\text{gen} \right)$, 我们通过梯度下降最小化损失 $L \left( \omega, \psi, \theta \right) = - \log p \left( \hat{u}_\text{gen} | x_\text{img}, u_\text{prompt} \right)$。

<!-- **Pretraining Dataset**. We limit our selection of pretraining data to datasets that are fully open-source (e.g., under permissive research licenses), and that have been used in prior work. Specifically, we use the LLaVa v1.5 data mixture, which consists of two subsets used for a multi-stage training pipeline. The first subset consists of a 558K sample mixture of examples sourced from various captioning datasets (e.g., Conceptual Captions, LAION Sharma et al., 2018; Schuhmann et al., 2021), while the second consists of 665K multimodal instruct tuning examples comprised of synthetic data generated in Liu et al. (2023c), as well as examples from existing vision-language training sets (e.g., GQA, TextCaps; Hudson & Manning, 2019; Sidorov et al., 2020), and notably, a sample of language-only data from ShareGPT (ShareGPT, 2023). We provide a comprehensive breakdown of the pretraining data mixture in §A.1. -->
**预训练数据集**。我们将预训练数据的选择限制在完全开源(例如, 在宽松的研究许可下)且已在先前工作中使用过的数据集。具体而言, 我们使用 LLaVa v1.5 数据混合, 它包含用于多阶段训练流水线的两个子集。第一个子集包含源自各种字幕数据集(例如, Conceptual Captions, LAION Sharma et al., 2018; Schuhmann et al., 2021)的 558K 示例样本混合, 第二个子集包含 Liu et al. (2023c) 生成的由合成数据组成的 665K 个多模式指令调整示例, 以及来自现有视觉-语言训练集 (例如, GQA, TextCaps; Hudson & Manning, 2019; Sidorov et al., 2020) 的示例, 尤其是来自 ShareGPT (ShareGPT, 2023) 的纯语言数据样本。我们在 $\S$ [A.1](#todo) 中提供了预训练数据混合的全面分解。

<!-- **Training Implementation & Verification**. To investigate the design axes enumerated in §1, we require code for VLM training that is efficient and flexible; critically, we need the ability to easily swap out vision and LM backbones and handle arbitrary optimization procedures (e.g., freezing the vision backbone during training). With these requirements, we implement our training codebase in PyTorch, using Fully Sharded Data Parallel (FSDP; Zhao et al., 2023) and BF16 mixed precision. FSDP lets us specify precision for individual model components (e.g., FP16 for vision backbones, BF16 for LMs), enables portability to different hardware, and provides minimal implementation overhead. Following reproducibility practices from prior work (Karamcheti et al., 2021; Biderman et al., 2023), we fix initialization randomness and fix batch order during training. We leverage TIMM (Wightman, 2019) and Hugging Face Transformers (Wolf et al., 2019) to provide pretrained models. -->
**训练实施和验证**。为了研究 $\S$ [1](#Introduction) 中列举的设计轴, 我们需要高效灵活的 VLM 训练代码; 至关重要的是, 我们需要能够轻松替换视觉和 LM backbones 并处理任意优化程序(例如, 在训练期间冻结视觉 backbone)。针对这些要求, 我们在 PyTorch 中实现了我们的训练代码库, 使用 Fully Sharded Data Parallel (FSDP; Zhao et al., 2023) 和 BF16 混合精度。FSDP 使我们指定单个模型组件的精度(例如, 视觉 backbones 为 FP16, LMs 为 BF16), 能够移植到不同的硬件, 并提供最小的实施开销。遵循先前工作 (Karamcheti et al., 2021; Biderman et al., 2023) 中的可重复性实践, 我们修复了初始化随机性, 并在训练期间修复了 batch order。我们利用 TIMM (Wightman, 2019) 和 Hugging Face Transformers (Wolf et al., 2019) 提供预训练模型。

<!-- To validate our code, we run an apples-to-apples reproduction of LLaVa v1.5 (Liu et al., 2023b) at both the 7B and 13B parameter scale. Successful reproduction results are in Fig. 4 (left). We find our implementation is considerably more efficient than the reference LLaVa v1.5 training implementation: when benchmarked on the same hardware (an AWS p4de.24xlarge node with 8 A100 GPUs), we observe 20% faster step times with our FSDP-backed implementation, a notable gain given LLaVa leverages the well-optimized DeepSpeed ZeRO library (Rasley et al., 2020). -->
为了验证我们的代码, 我们在 7B 和 13B 参数规模上运行了 LLaVa v1.5 (Liu et al., 2023b) 的同类复制。成功的复现结果如[图4](#fig.4)(左)所示。我们发现我们的实现比参考的 LLaVa v1.5 训练实现效率高得多: 在相同硬件(具有 8 个 A100 GPU 的 AWS p4de.24xlarge 节点)上进行基准测试时, 我们观察到使用 FSDP 支持的实现, step 时间加快 20%, 鉴于 LLaVa 利用了充分优化的 DeepSpeed ZeRO 库 (Rasley et al., 2020), 这是一个显著的进步。

todo: 图4
<!-- Figure 4. Reproducing LLaVa v1.5 & Exploring Optimization Procedures. To validate our training codebase ($\S$ [2](#Preliminaries)), we reproduce LLaVa v1.5 (green), with our models reproducing the performance reported in Liu et al. (2023b) (gray). We then run our first experiment ($\S$ [4.1](#Optimization-Procedure)) investigating the need for expensive multi-stage training (right). We find that single-stage training produces VLMs that maintain or outperform multi-stage models (orange), saving considerable compute; as a result, we carry this change forward to all future experiments. -->
图4. 复现 LLaVa v1.5 并探索优化程序。为了验证我们的训练代码库 ($\S$ [2](#Preliminaries)), 我们复现了 LLaVa v1.5 (绿色), 我们的模型复现了 Liu et al. (2023b) 中报告的性能(灰色)。然后, 我们运行我们的第一个实验 ($\S$ [4.1](#Optimization-Procedure)), 调查昂贵的多阶段训练的必要性(右)。我们发现单阶段训练生成的 VLMs 保持或超越多阶段模型(橙色), 节省大量计算; 因此, 我们将这一变化延续到所有未来的实验中。

<!-- **We highlight this open-source training codebase as one of the key contributions of this work**. Unlike other open codebases, we provide a modular and expressive interface for easily specifying or adding model components, optimization procedures, and data with minimal code changes (Fig. 2; right). In providing an efficient and easily extensible framework, we enable future research around designing new evaluations, developing and training new VLMs, and finetuning or otherwise adapting existing models for diverse downstream applications – all while maintaining a high standard of reproducibility and controlled experimentation. -->
**我们强调这个开源训练代码库是这项工作的主要贡献之一**。与其他开源码库不同, 我们提供了一个模块化且富有表现力的接口, 只需最少的代码更改即可轻松指定或添加模型组件、优化程序和数据([图2](#fig.2); 右)。通过提供高效且易于扩展的框架, 我们支持(围绕设计新评估、开发和训练新 VLMs 以及微调或以其他方式调整现有模型以适应各种下游应用的)未来研究——同时保持高标准的可重复性和受控实验。

# Evaluation Suite
<!-- The first contribution of this work is a unified evaluation suite that offers fine-grained insight into the capabilities of a given VLM. Recent work in evaluating VLMs tends to rely on automated evaluations that use powerful LMs such as GPT-4 (OpenAI et al., 2023) to judge relative and subjective performance(Liu et al., 2023e; Yu et al., 2023), making it hard to measure the absolute impact of a given design change. Instead, we focus on evaluations with well-defined metrics, spanning the following three areas: -->
这项工作的第一个贡献是统一的评估套件, 它提供对给定 VLM 能力的细粒度洞察。近期评估 VLMs 的工作趋向依赖于自动化评估, 该评估使用强大的 LMs, 例如 GPT-4 (OpenAI et al., 2023), 来判断相对和主观的性能 (Liu et al., 2023e; Yu et al., 2023), 使得难以衡量给定设计变更的绝对影响。相反, 我们聚焦于具有明确定义的指标的评估, 涵盖以下三个领域:

<!-- *Open-Ended Visual Question Answering*. We evaluate on VizWiz (Bigham et al., 2010), VQAv2 (Goyal et al., 2017), GQA (Hudson & Manning, 2019), and TextVQA (Singh et al., 2019). Both VizWiz and VQAv2 assess general visual reasoning; VizWiz also contains a series of unanswerable questions. GQA evaluates spatial reasoning, while TextVQA assesses reasoning around text (e.g., labels, signage) present in an image. -->
*开放式视觉问答*。我们在VizWiz (Bigham et al., 2010), VQAv2 (Goyal et al., 2017), GQA (Hudson & Manning, 2019) 和 TextVQA (Singh et al., 2019)上进行评估。VizWiz 和 VQAv2 评估通用视觉推理; VizWiz 还包含一系列无法回答的问题。GQA 评估空间推理, 而 TextVQA 评估围绕图像中出现的文本(例如标签、指示牌)的推理。

<!-- *Localization*. Part of the pretraining data mixture (from $\S$ [2](#Preliminaries)) contains examples of predicting normalized bounding box coordinates given referring expressions in language. As such, we evaluate bounding box prediction accuracy on RefCOCO, RefCOCO+, and RefCOCOg (Kazemzadeh et al., 2014; Yu et al., 2016), and on OCID-Ref (Wang et al., 2021). RefCOCO focuses on short descriptions with spatial anchors, RefCOCO+ on strictly appearance based descriptions, and RefCOCOg on long, rich descriptions; OCID-Ref is a robotics dataset probing out-of-distribution generalization, with a focus on localizing objects in clutter. -->
*定位*。部分预训练数据混合物(来自 $\S$ [2](#Preliminaries))包含在给定语言中的参考表达式的情况下预测标准化边界框坐标的示例。因此, 我们在 RefCOCO, RefCOCO+, RefCOCOg (Kazemzadeh et al., 2014; Yu et al., 2016) 和 OCID-Ref (Wang et al., 2021) 上评估了边界框预测准确性。RefCOCO 聚焦于具有空间锚点的简短描述, RefCOCO+ 聚焦于严格基于外观的描述, RefCOCOg 聚焦于长而丰富的描述; OCID-Ref 是一个探索分布外泛化的机器人数据集, 聚焦于定位杂乱中的物体。

<!-- *Challenge Sets (Closed-Set Prediction)*. We evaluate on Visual Spatial Reasoning (VSR; Liu et al., 2022), TallyQA (Acharya et al., 2018), POPE (Li et al., 2023d), and AI2 Diagrams (AI2D; Kembhavi et al., 2016). VSR consists of challenging True/False questions about individual spatial relationships in diverse scenes (e.g., "the cake is at the edge of the dining table"); this is an especially challenging task, with most existing models failing to outperform the majority class baseline (51%). TallyQA consists of questions that assess a VLM's ability to count objects described in language, with expressions that range in complexity. POPE consists of targeted Yes/No questions that assess a VLM's propensity to hallucinate. Finally, AI2D consists of multiple choice questions about scientific diagrams and charts, many of which require reading labels or text annotations (e.g., flowchart labels, plot axes). -->
*挑战集(闭集预测)*。我们在 Visual Spatial Reasoning (VSR; Liu et al., 2022), TallyQA (Acharya et al., 2018), POPE (Li et al., 2023d) 和 AI2 Diagrams (AI2D; Kembhavi et al., 2016) 上进行评估。VSR 包括(关于不同场景中个体空间关系的)具有挑战性的真/假问题(例如, "蛋糕在餐桌边缘"); 这是一项特别具有挑战性的任务, 大多数现有模型都无法超越多数类基线(51%)。TallyQA 包含一些问题, 用于评估 VLM 统计用语言描述的对象的能力, 其表达方式的复杂程度各不相同。POPE 包括有针对性的是/否问题, 这些问题评估 VLM 对幻觉的倾向。最后, AI2D 包含有关科学图表的多选择问题, 其中许多问题需要阅读标签或文本注释(例如, 流程图标签、绘图轴)。

<!-- We use the validation sets for all benchmarks except GQA (where use the recommended the test-dev split), VSR (where we use the zero-shot test split), and POPE (where there is only a single evaluation split). We provide further detail around evaluation protocols in [Appx. B](#todo). -->
我们对除 GQA (使用推荐的测试开发 split)、VSR (使用零样本测试 split) 和 POPE (只有一个评估 split) 之外的所有基准测试, 都使用验证集。我们在 [附录 B](#todo) 中提供了有关评估协议的更多详细信息。

# Experiments – Investigating Design Axes
<!-- Our second contribution is a series of targeted experiments exploring the VLM design space along four key axes: ($\S$ [4.1](#Optimization-Procedure)) optimization procedure, ($\S$ [4.2](#Image-Processing-&-Visual-Representations)) image processing and visual representations, ($\S$ [4.3](#Integrating-Language-Models)) language models, and ($\S$ [4.4](#Scaling-Properties:-Training-Time-&-Data)) scaling properties such as training time and data diversity. -->
我们的第二个贡献是一系列有针对性的实验, 沿着四个关键轴探索 VLM 设计空间: ($\S$ [4.1](#Optimization-Procedure)) 优化程序, ($\S$ [4.2](#Image-Processing-&-Visual-Representations)) 图像处理和视觉表示, ($\S$ [4.3](#Integrating-Language-Models)) 语言模型, ($\S$ [4.4](#Scaling-Properties:-Training-Time-&-Data)) 缩放属性, 例如训练时间和数据多样性。

<!-- **Experiment Design: Protocols & Drawing Conclusions**. We first validate our VLM training implementation by reproducing LLaVa v1.5 (see $\S$ [2](#Preliminaries)), adopting the design choices of the original work – the same choices used by many other recent VLMs: "letterbox padding" to process images, CLIP ViT-Large with a patch size of 14 and input resolution of 336px (CLIP ViT-L/14 @ 336px; Radford et al., 2021) as the visual representation, Vicuna v1.5 as the LM backbone, and the two-stage training pipeline using both data subsets described in $\S$ [2](#Preliminaries). Successful reproduction results at both the 7B and 13B scale are in Fig. 4 (left). Given both the fidelity of our reproduction and the prevalence of these design choices, we anchor our analyses around this parameterization. Critically, each of the experiments in $\S$ [4.2](#Image-Processing-&-Visual-Representations), $\S$ [4.3](#Integrating-Language-Models), and $\S$ [4.4](#Scaling-Properties:-Training-Time-&-Data) are formulated as *single-step changes of this base architecture, with all other choices held constant*. -->
**实验设计: 协议和总结**。我们首先通过复现 LLaVa v1.5 (参见 $\S$ [2](#Preliminaries)), 验证我们的 VLM 训练实现, 采用原始工作的设计选择——许多其他最近的 VLMs 使用的相同选择: "letterbox 填充" 以处理图像, CLIP ViT-Large 作为视觉表示, 其 patch 大小为 14, 输入分辨率为 336px (CLIP ViT-L/14 @ 336px; Radford et al., 2021), Vicuna v1.5 作为 LM backbone, 和使用 $\S$ [2](#Preliminaries) 中描述的数据子集的两阶段训练流水线。7B 和 13B 规模的成功复现结果如[图4](#fig.4)(左)所示。考虑到我们复现的保真度和这些设计选择的普遍性, 我们围绕这个参数化进行分析。至关重要的是, $\S$ [4.2](#Image-Processing-&-Visual-Representations)、$\S$ [4.3](#Integrating-Language-Models) 和 $\S$ [4.4](#Scaling-Properties:-Training-Time-&-Data) 中的每个实验都被表述为 *此基础架构的单步改变, 所有其他选择保持不变*。

<!-- As each evaluation in $\S$ [3](#Evaluation-Suite) uses different metrics with different scales, direct comparison is challenging. We address this by computing normalized Z-scores for each model and evaluation (using the mean and standard deviation across all models). These scores are used to compute statistical significance (further details in $\S$ [B.2](#todo)), and to set the relative scales of each radar plot (for completeness, we also provide the absolute metrics as colored and bolded labels). -->
由于 $\S$ [3](#Evaluation-Suite) 中的每个评估都使用具有不同尺度的不同指标, 直接比较具有挑战性。我们通过计算每个模型和评估的归一化 Z-分数(使用所有模型的平均值和标准差)来解决这个问题。这些分数用于计算统计显着性(更多详细信息请参阅 $\S$ [B.2](#todo)), 并设置每个雷达图的相对尺度(为了完整起见, 我们还以彩色和粗体标签的形式提供绝对指标)。

## Optimization Procedure
<!-- In this section we focus on design choices around the optimization procedure used to initialize and train each of the three components described in $\S$ [2](#Preliminaries). Specifically, we examine the effects of multi-stage training where different VLM components are frozen at different points in training. -->
在本节中, 我们专注于围绕优化程序的设计选择, 用于初始化和训练$\S$ [2](#Preliminaries)中描述的三个组件中的每一个。具体而言, 我们研究了多阶段训练的效果, 其中不同的 VLM 组件在训练的不同点被冻结。

<!-- **Multi-Stage Training**. One of the prevalent design choices adopted by many VLMs (Chen et al., 2023a; Ye et al., 2023) is the inclusion of a two-stage training pipeline: (1) an alignment stage to align vision and language features by training the randomly initialized projector $F_\psi$ in isolation, freezing all other components (Fig. 4, right) and (2) a finetuning stage, where only the visual representation is frozen while both the projection and LM are trained. -->
**多阶段训练**。许多 VLMs (Chen et al., 2023a; Ye et al., 2023) 采用的流行设计选择之一是两阶段训练流水线: (1) 对齐阶段, 通过单独训练随机初始化的投影器 $F_\psi$ 来对齐视觉和语言特征, 冻结所有其他组件([图4](#fig.4), 右); (2) 微调阶段, 其中仅冻结视觉表征, 同时训练投影和 LM。

<!-- Adopting multi-stage training complicates implementation and adds to training cost; therefore, as an initial experiment, we evaluate the need for this first stage through a targeted ablation. We compare the default two-stage training procedure with a single-stage approach that skips directly to finetuning $F_\psi$ and the LM. We find (Fig. 4; left) that including the explicit projector pretraining stage is unnecessary, with single-stage training improving aggregate performance ($p = 0.00558$). Eliminating this first stage saves 20-25% of training cost, and removes the need for additional, stage-specific data (e.g., the captioning subset from $\S$ [2](#Preliminaries)). *As this change strictly improves performance and efficiency, we adopt single-stage training for all following experiments.* -->
采用多阶段训练使实施复杂化并增加训练成本; 因此, 作为初步实验, 我们通过有针对性的消融, 评估第一阶段的必要性。我们将默认的两阶段训练程序与(直接跳到微调 $F_\psi$ 和 LM 的)单阶段方法进行比较。我们发现包括精密投影器预训练阶段是不必要的([图4](#fig.4); 左), 单阶段训练提高总体性能($p = 0.00558$)。消除第一阶段节省 20-25% 的训练成本, 并且无需额外的特定阶段数据(例如, 来自 $\S$ [2](#Preliminaries) 的字幕子集)。*由于这一变化确实提高了性能和效率, 因此我们对所有后续实验都采用单阶段训练*。

<!-- **Full Finetuning through Visual Backbones**. Another popular design choice in existing VLMs that leverage pretrained visual representations is to leave the visual backbone frozen during the entirety of training (Liu et al., 2023b; Driess et al., 2023; Li et al., 2023b). Such a choice limits the potential to learn improved visual representations conducive to language generation during the course of training. Thus, we ask – *is there potential to improve VLM performance by finetuning the full model, including the visual backbone*? We find (Fig. 5) that this is not the case, and that finetuning the visual backbone significantly degrades performance ($p = 0.00381$), especially on tasks requiring fine-grained spatial reasoning such as RefCOCO and OCID-Ref. -->
**通过视觉 Backbones 进行完全微调**。现有的(利用预训练视觉表征的) VLMs 的另一种流行设计选择是在整个训练期间冻结视觉 backbone (Liu et al., 2023b; Driess et al., 2023; Li et al., 2023b)。这样的选择限制了在训练过程中学习(有利于语言生成的)改进视觉表征的潜力。因此, 我们要问——*通过微调整个模型, 包括视觉 backbone, 是否有可能提高 VLM 性能?* 我们发现事实并非如此([图5](#fig.5)), 并且微调视觉 backbone 显著降低性能($p = 0.00381$), 尤其是在需要细粒度空间推理的任务上, 例如 RefCOCO 和 OCID-Ref。

todo: 图5

<!-- *Remark*. The degraded performance from full finetuning could be for a number of reasons ranging from the scale and diversity of the vision-language data we train on to language generation as a learning objective (vs. objectives that encourage learning fine-grained perceptual features). Especially given the existence of closed-source models such as Fuyu-8B (AI, 2023) that adopt this paradigm to great success, we believe that identifying ways to prevent such feature collapse during VLM training (e.g., via auxiliary objectives) to be a rich direction for future work. -->
*注释*。完全微调导致的性能下降可能有多种原因, 从我们训练用的视觉-语言数据的规模和多样性, 到将语言生成作为学习目标(vs. 鼓励学习细粒度感知特征的目标)。特别是考虑到闭源模型的存在, 例如 Fuyu-8B (AI, 2023), 它采用这种模式取得巨大成功, 我们认为, 确定在 VLM 训练期间防止此类特征崩溃的方法(例如, 通过辅助目标)将成为未来工作的丰富方向。

## Image Processing & Visual Representations
<!-- **Choosing a Pretrained Vision Representation**. CLIP (Radford et al., 2021) has become the default choice for visual representation for almost all VLMs, despite a wealth of visual representations trained on diverse data sources. In this experiment, we perform a head-to-head comparison between CLIP, SigLIP (Zhai et al., 2023), DINOv2 (Oquab et al., 2023), and a standard Vision Transformer pretrained for classification (on ImageNet-21K, finetuned on ImageNet-1K; Dosovitskiy et al., 2021; Steiner et al., 2021); for fair comparison, we use the ViT-Large model variant.<sup>[2](#superscript.2)</sup> We find (Fig. 6; left) that the backbones trained with vision-language contrastive objectives (i.e., CLIP, SigLIP) are significantly more performant than alternatives ($p = 7.11e-8$). -->
**选择预训练的视觉表征**。CLIP (Radford et al., 2021)已成为几乎所有 VLMs 的视觉表征的默认选择, 尽管在不同的数据源上训练了丰富的视觉表征。在本实验中, 我们对 CLIP, SigLIP (Zhai et al., 2023), DINOv2 (Oquab et al., 2023) 和预训练用于分类的标准 Vision Transformer (在 ImageNet-21K 上训练, 在 ImageNet-1K 上微调; Dosovitskiy et al., 2021; Steiner et al., 2021) 进行了正面比较; 为了公平比较, 我们使用 ViT-Large 模型变体<sup>[2](#superscript.2)</sup>。我们发现([图6](#fig.6); 左)使用视觉-语言对比目标(即 CLIP、SigLIP)训练的 backbones 比替代方案性能显着提高($p = 7.11e-8$)。

<!-- To evaluate on an image resolution common to all representations (224px), we use the shape-optimized SigLIP model (ViT-SO Alabdulmohsin et al., 2023) that is slightly larger than a ViT-Large at 400M parameters (vs 307M). -->
> <sup>2</sup><span id='superscript.2'></span>评估所有表征通用的图像分辨率 (224px), 我们使用形状优化的 SigLIP 模型 (ViT-SO Alabdulmohsin et al., 2023), 该模型比 400M 参数的 ViT-Large 略大 (vs 307M)。

<!-- *Remark*. While the vision-language contrastive objective is one explanation for the strengths of CLIP and SigLIP, another possible explanation is one of training image distribution. Both CLIP and SigLIP contain internet-sourced images (e.g., sketches, diagrams, animated graphics, etc.) not in ImageNet or in the DINOv2 pretraining data. -->
*注释*。尽管视觉-语言对比目标是 CLIP 和 SigLIP 优势的一个解释, 但另一个可能的解释是训练图像分布。CLIP 和 SigLIP 都包含来自互联网的图像(例如草图、图表、动画图形等), 这些图像不在 ImageNet 或 DINOv2 预训练数据中。

**Image Processing across Visual Backbones**. Most images have resolutions and aspect ratios that widely vary, yet most visual backbones expect square images at a fixed size; to reconcile this, the overwhelming default is to "resize & crop" an image to size. While this tends to work well for applications such as classification, cropping out parts of an image is especially harmful for tasks requiring full-scene reasoning. In this experiment, we evaluate three different image processing schemes – the default "resize & crop" scheme, the "letterbox padding" scheme used by LLaVa v1.5 that pads non-square images to square, and a "naive resize" scheme that warps the original image aspect ratio, squeezing or stretching an image to square. Our findings (Fig. 6; middle) are surprising: while cropping is clearly suboptimal, the "naive resize" scheme is the most performant for CLIP. For SigLIP, both "naive resize" and "letterbox padding" perform similarly. In general, our results favor "naive resizing" over "letterbox padding" but we cannot rule the improvement statistically significant ($p = 0.0176$).


*Remark*. Two speculative arguments for naively resizing an image over padding are those of minimizing "dead pixels" and distribution shift. An image with a 16:9 aspect ratio that is padded to square introduces a large amount of uninformative pixels (exceeding 40%); warping the aspect ratio is possibly less of a shift. Coupled with the innate patch dimensionality of a Vision Transformer ($d = 1024$ for a $16 \times 16$ pixel patch), naively resizing an image may preserve enough information for the downstream LM (with 7B+ parameters) to extract the properties necessary for downstream tasks.


**Scaling Image Resolution**. Another trend in recent VLMs is increasing input image resolution with the hope of capturing fine-grained details that improve downstream performance (Liu et al., 2023b; Li et al., 2023a). Our findings (Fig. 6; right) confirm this hypothesis, with scaling to 336px or 384px offering significant improvements ($p = 6.05e-4$).


*Remark*. While scaling up image resolution seems like a clear win, we caution that it comes with a significant increase in compute complexity for VLMs that project individual ViT patches into the embedding space of an LM. Assuming a fixed patch granularity, doubling the input resolution results in four times the number of input patches fed to the LM. Coupled with the quadratic cost of traditional Transformer attention as a function of sequence length, this is a sixteen-fold increase in time complexity (with a comparable explosion in memory requirements).


**Ensembling Different Visual Representations**. A rich body of prior work in vision identifies that different types of visual representations trained with different inductive biases can lead to improved performance for a broad spectrum of applications (Kobayashi et al., 2022; Karamcheti et al., 2023). Motivated by this, we ask if this same trend holds true for VLM training – specifically whether ensembling DINOv2 features with vision-language contrastive features from CLIP and SigLIP can lead to improved performance, following the approach taken in Kerr et al. (2023). To implement this efficiently, we simply concatenate patch features from different backbones along the channel dimension for each patch, resulting in the same number of input patch embeddings, just with double the feature dimension. To adjust for this, we just increase the input dimension to our projector $F_\psi$ (a 2-layer MLP) at negligible cost. We find (Fig. 7 - left) that fusing DINOv2 and SigLIP features provides significant gains across the board ($p = 0.00164$), with a notable exception for the DINOv2 + CLIP models ($p = 0.37313$), where combining DINOv2 features seem to be particularly harmful on TextVQA. Looking at the remaining results, we see especially impressive gains of 5-10% on localization and challenge tasks; in general, the DINOv2 + SigLIP fused representations are the most performant visual representations we try, with virtually no added parameters.


*Remark*. Following the hypotheses in Kerr et al. (2023) and similar work, we believe that DINOv2 features provide features that capture low-level spatial properties of an image, augmenting the higher-level "semantic" properties captured by vision-language contrastive models. We note that this conclusion may generalize beyond DINO-style backbones; the only reason we do not evaluate the fusion of ImageNet and CLIP/SigLIP backbones as well is due to a mismatch in patch granularity (the ImageNet backbone uses a patch granularity of $16 \times 16$ vs. the $14 \times 14$ granularity used by all other backbones). We believe that further exploring the impact on these type of fused, multi-resolution features for VLMs is a compelling avenue for future work.

##  Integrating Language Models
Base vs. Instruct-Tuned LMs. Instruct tuning (or chat
tuning; Ouyang et al., 2022; Chung et al., 2022) is a way
to finetune base LMs (trained for next-token prediction) to
behave as dialogue agents, offering a natural input/output
interface for a wide spectrum of applications. As a result,
instruct tuned models like Vicuna ( ˜ Zheng et al., 2023) have
become the default backbone for VLMs. Unfortunately,
instruct tuning has drawbacks, introducing bias and regressions in performance (Ouyang et al., 2022). Thus, in this
experiment we evaluate the impact of instruct-tuned LM
backbones on downstream VLM performance via a head-to-head comparison between a base LM (Llama-2; Touvron
et al., 2023), and an instruct-tuned variant (Vicuna v1.5). 
We find (Fig. 7 - right) that instruction-tuned LMs yield no
statistically significant improvement in performance over
base LMs (p = 0.34854), but differ in qualitative performance. Specifically, we observe that instruct-tuned LMs
lead to VLMs that are more verbose, prone to hallucination,
and generally less specific in their responses (Fig. 11).


Do Better LMs Lead to Better VLMs? We investigate how
LM performance on language-only benchmarks translates to
downstream VLM performance, training VLMs from Mistral v1 7B and Mistral Instruct v1 7B (Jiang et al., 2023),
recent LMs that outperform Llama-2 on language and code
benchmarks (Hendrycks et al., 2021; Chen et al., 2021). We
find (Fig. 12) that these VLMs are not significantly more per formant than VLMs trained from Llama-2 (p = 0.03097);
an exciting avenue for future work is investigating how LM
pretraining mixtures correlate with VLM performance.


Co-training on Language-only Safety Data. The LLaVa
v1.5 pretraining dataset we use for training consists of 40K
examples of language-only data sourced from ShareGPT
(ShareGPT, 2023); this data consists of a diverse set of
user-uploaded conversations with OpenAI’s ChatGPT; crucially many of the examples in this dataset contain toxic,
inappropriate, or otherwise unsafe inputs, and the corresponding “guarded” responses from ChatGPT (e.g., “as an
AI, I cannot comment on...”). In this experiment, we ablate the impact of co-training on this language-only data
on downstream performance, with a goal of understanding
if adding language-only data unrelated to visual reasoning
hurts performance relative to training on multimodal data
alone. We find (Fig. 8; left) that removing language-only
data only slightly improves performance (p = 0.13655).


However, given that the language-only data is the only
source of “safety” data during finetuning, we explicitly
probe our VLMs with directly offensive and toxic prompts,
to evaluate how important this data is for inducing safeguards on VLM outputs. In our adversarial testing, we find
that especially for VLMs trained from base LMs such as
Llama-2, including this co-training data is important for
inducing at least a minimal set of safeguards; Fig. 8 demonstrates the importance of co-training on VLM generations
when prompted with questions with direct racist intent.


Remark. We focus our probing mostly around unsafe responses around racism, xenophobia, and gender bias. These
biases are also prevalent in language, and explicitly represented in the ShareGPT co-training data. We address
VLM-specific harms in our discussion of broader impacts.


## Scaling Properties: Training Time & Data
In this section, we investigate how existing VLMs scale with
training time and added data; critically, we examine choices
of training time (are we undertraining our models), and how
adding diverse datasets impacts downstream performance.


Are we Undertraining? We explore the impact of training
time as a function of training epochs. Unlike existing VLMs
like PaLI or LLaVa that perform at most a single epoch, we
compare performance when training at different numbers
of epochs. We find (Fig. 10; middle) evidence of severe
underfitting with a single epoch, with steady improvement
(especially for tasks requiring structured output such as
RefCOCO) until two epochs, when performance plateaus.
We find that training for two epochs yields a significant
improvement over training for one epoch (p = 0.00496).


Adding Additional Vision-Language Data. We identify
two recently proposed datasets: LVIS-Instruct-4V (Wang
et al., 2023), obtained by prompting GPT-4V to generate
rich synthetic examples from images sourced from LVIS
(Gupta et al., 2019), and LRV-Instruct (Liu et al., 2023a)
that specifically optimizes for image diversity relative to
existing datasets (adding e.g., charts, scientific diagrams,
and news printings). We find (Fig. 10; right) that adding
both datasets improves performance (p = 0.01459), but that
LRV-Instruct has a larger impact, indicating the importance
of diverse images for scaling future VLMs.

In [ ]:
from pypdf import PdfReader

pdf_path = "/mnt/d/github/paper/具身机器人/Prismatic VLMs_Investigating the Design Space of Visually-Conditioned Language Models.pdf"

reader = PdfReader(pdf_path)
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()
print(text)